## 算分公式開發過程

### submit example

```
ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0050,1,87.15,1,88.05,-1,88.00,-1,87.25,1,87.45
0051,1,32.49,1,33,1,32.72,1,32.75,-1,32.44
0052,-1,57.75,1,58.5,1,59.8,-1,59,-1,57.9
0053,1,37.38,1,37.92,-1,37.44,-1,37.2,1,37.44
0054,1,24.63,1,24.94,-1,24.05,1,24.63,1,24.7
0055,-1,17.1,0,17.1,-1,17,0,17,1,17.02
0056,-1,26,1,26.25,-1,26.17,1,26.32,1,26.45
0057,1,52.4,1,52.9,-1,52.6,1,52.9,-1,52.05
0058,1,46.56,1,47.02,1,47.27,0,47.27,1,47.37
0059,-1,41.9,1,42.07,-1,41.44,0,41.44,1,41.9
006201,-1,14.98,1,15.27,-1,15.00,-1,14.97,-1,14.90
006203,1,40.24,1,40.84,-1,40.27,1,40.42,-1,40.40
006204,1,55.35,1,55.70,-1,55.35,1,55.40,1,55.45
006208,1,49.75,1,49.8,1,49.85,1,49.11,-1,49.35
00690,1,22.48,1,22.44,-1,22.24,1,22.28,-1,22.42
00692,1,22.14,1,22.23,-1,22.04,1,22.12,1,22.19
00701,1,21.46,1,21.5,-1,21.38,1,21.41,0,21.41
00713,-1,31.22,-1,31.1,-1,31.07,1,31.14,-1,31.12
```

In [1]:
import csv
import pandas as pd
import numpy as np
from collections import OrderedDict

### etf ID list

In [2]:
etfid_list = ['0051', '0050', '0052', '0053', '0054', '0055', '0056', '0057', 
              '0058', '0059', '006201', '006203', '006204', '006208', '00690', 
              '00692', '00701', '00713']

In [24]:
week_colunm_list = ['Mon_ud', 'Mon_cprice', 'Tue_ud', 'Tue_cprice', 'Wed_ud',
 'Wed_cprice', 'Thu_ud', 'Thu_cprice', 'Fri_ud', 'Fri_cprice']

In [39]:
ETF_NUM = len(etfid_list)
SUBMIT_COLNUM = len(week_colunm_list) + 1

### 計算一週單一個股的預測分數 (暫時不用)

In [5]:
### assume this are 0050 real values in a week ###
actual_0050 = OrderedDict({'20180122': (1, 87.15), '20180123': (1, 88.05), '20180124': (-1, 88.00), 
                '20180125': (-1, 87.25), '20180126': (1, 87.45)})

### assume this are 0050 predict values in a week ###
predict_0050 = OrderedDict({'20180122': (1, 87.45), '20180123': (1, 88.05), '20180124': (1, 88.3), 
                '20180125': (-1, 87.25), '20180126': (1, 88.7)})

In [20]:
def evaluate_temp(predict, actual):
    """ description 
      - 假設一週必定是 5 天，如果當週有放假日，可先在當天塞假資料 (ex. 都是 1 之類的)
      - score 的滿分是 1，但可能負無限大。如程式日期或天數發生錯誤，回傳 1000 表示 error
    """
    wrighting = [0.1, 0.15, 0.2, 0.25, 0.3]
    score = 0
    if predict.keys() == actual.keys() and len(predict) == 5:
        for i, j in enumerate(actual):
            day_score = 0.0
            if predict[j][0] == actual[j][0]:
                day_score += 0.5
            
            day_score += (((actual[j][1] - abs(predict[j][1] - actual[j][1]))/actual[j][1])*0.5) 
            score += day_score * wrighting[i]
    else:
        score = 1000
        
    return score
    
evaluate_temp(predict_0050, actual_0050)

0.8973428915367424

### 讀取官網範例 submit 格式

In [7]:
answer = pd.read_csv('../data/submit_example.csv', dtype={'ETFid' : np.str})

In [8]:
list(answer.columns)

['ETFid',
 'Mon_ud',
 'Mon_cprice',
 'Tue_ud',
 'Tue_cprice',
 'Wed_ud',
 'Wed_cprice',
 'Thu_ud',
 'Thu_cprice',
 'Fri_ud',
 'Fri_cprice']

In [9]:
predict = pd.read_csv('../data/submit_example.csv', dtype={'ETFid' : np.str})
answer = pd.read_csv('../data/answer_example.csv', dtype={'ETFid' : np.str})

In [18]:
test1 = OrderedDict([('Mon_ud', 1),
             ('Mon_cprice', 87.150000000000006),
             ('Tue_ud', 1),
             ('Tue_cprice', 88.049999999999997),
             ('Wed_ud', -1),
             ('Wed_cprice', 88.0),
             ('Thu_ud', -1),
             ('Thu_cprice', 87.25),
             ('Fri_ud', 1),
             ('Fri_cprice', 87.450000000000003)])

test2 = OrderedDict([('Mon_ud', 1),
             ('Mon_cprice', 32.490000000000002),
             ('Tue_ud', 1),
             ('Tue_cprice', 33.0),
             ('Wed_ud', 1),
             ('Wed_cprice', 32.719999999999999),
             ('Thu_ud', 1),
             ('Thu_cprice', 32.75),
             ('Fri_ud', -1),
             ('Fri_cprice', 32.439999999999998)])

In [63]:
def evaluate(predict: OrderedDict, actual: OrderedDict) -> float:
    wrighting = [0.1, 0.15, 0.2, 0.25, 0.3]
    prefix = ['Mon','Tue','Wed','Thu','Fri']
    score = 0
    
    if set(predict.keys()) != set(week_colunm_list) or set(actual.keys()) != set(week_colunm_list):
        raise Exception('Wrong columns for the data to be evaluated.')
    
    for _day in prefix:
        day_score = 0.0
        if predict[_day + '_ud'] == actual[_day + '_ud']:
            day_score += 0.5
            
        day_score += (((actual[_day + '_cprice'] - abs(predict[_day + '_cprice'] - actual[_day + '_cprice']))/actual[_day + '_cprice'])*0.5) 
        score += day_score * wrighting[prefix.index(_day)]
        
    return score
    
evaluate(test1, test2)

-0.21555764789856463

In [64]:
def evaluate_all(predict: pd.DataFrame, answer: pd.DataFrame) -> list:
    predict_dict = {}
    answer_dict = {}
    result = []
    
    ## check result format in a week
    if predict.shape != (ETF_NUM, SUBMIT_COLNUM) or answer.shape != (ETF_NUM, SUBMIT_COLNUM):
        raise Exception('Wrong dataframe size for a week.')
        
    for row_index in range(predict.shape[0]):
        this_predict = OrderedDict(predict.iloc[row_index])
        this_answer = OrderedDict(answer.iloc[row_index])
        ETFid = this_predict.pop('ETFid')
        predict_dict[ETFid] = this_predict
        
        ETFid = this_answer.pop('ETFid')
        answer_dict[ETFid] = this_answer
    
    if set(predict_dict.keys()) != set(etfid_list) or set(answer_dict.keys()) != set(etfid_list):
        raise Exception('Wrong ETF id list.')
    
    for ETFid in etfid_list:
        this_score = evaluate(predict_dict[ETFid], answer_dict[ETFid])
        result.append({'code': ETFid, 'score': this_score})  
        
    return result

In [65]:
evaluate_all(predict, answer)

[{'code': '0051', 'score': 1.0},
 {'code': '0050', 'score': 1.0},
 {'code': '0052', 'score': 1.0},
 {'code': '0053', 'score': 1.0},
 {'code': '0054', 'score': 1.0},
 {'code': '0055', 'score': 1.0},
 {'code': '0056', 'score': 0.99857142857142867},
 {'code': '0057', 'score': 0.99904942965779475},
 {'code': '0058', 'score': 1.0},
 {'code': '0059', 'score': 1.0},
 {'code': '006201', 'score': 1.0},
 {'code': '006203', 'score': 1.0},
 {'code': '006204', 'score': 1.0},
 {'code': '006208', 'score': 1.0},
 {'code': '00690', 'score': 1.0},
 {'code': '00692', 'score': 1.0},
 {'code': '00701', 'score': 1.0},
 {'code': '00713', 'score': 0.84749357326478147}]